 <h2> Domain.com.au Web Scraping</h2>
                             <img src="http://www.domain.com.au/Content/Files/misc/domainLogoHead.gif">

IMPORTING LIBRARIES

In [1]:
# HTML extraction & Parsin
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib
import math
import os
from constants import Brisbane_suburbs,Melbourne_suburbs,Sydney_suburbs,Adelaide_suburbs,Perth_suburbs



# Calculate Latitude & Longitude
from geopy.geocoders import Nominatim
# Calculate Distance from Latitude & Longitude
from math import sin, cos, sqrt, atan2, radians
import emailsend



SET GLOBAL PARAMETERS

In [2]:
urls = []
# Change the index to change the city
city_list = ['Sydney','Melbourne','Brisbane','Adelaide','Perth']
city = city_list[1]
if city == 'Sydney':
    suburbs_list = Sydney_suburbs
elif city == 'Melbourne':
    suburbs_list = Melbourne_suburbs
elif city == 'Brisbane':
    suburbs_list = Brisbane_suburbs
elif city == 'Adelaide':
    suburbs_list = Adelaide_suburbs
elif city == 'Perth':
    suburbs_list = Perth_suburbs



print(f"CHOSEN CITY: {city}")
if not os.path.exists(f"../MLrealestate/data/{city}/"):
    os.mkdir(f"../MLrealestate/data/{city}/")
print(suburbs_list)
for suburb in suburbs_list:
    urls.append(f"https://www.domain.com.au/sold-listings/{suburb}/?excludepricewithheld=1&landsize=10-any&ssubs=0&page=")

house_list = []



CHOSEN CITY: Melbourne
['lalor-vic-3075', 'lancefield-vic-3435', 'lang-lang-east-vic-3984', 'langwarrin-vic-3910', 'langwarrin-south-vic-3911', 'launching-place-vic-3139', 'laverton-vic-3028', 'laverton-north-vic-3026', 'lilydale-vic-3140', 'long-forest-vic-3340', 'lower-plenty-vic-3093', 'lynbrook-vic-3975', 'lyndhurst-vic-3975', 'lysterfield-vic-3156', 'lysterfield-south-vic-3156', 'macclesfield-vic-3782', 'macedon-vic-3440', 'macleod-vic-3085', 'maddingley-vic-3340', 'maidstone-vic-3012', 'main-ridge-vic-3928', 'malvern-vic-3144', 'malvern-east-vic-3145', 'mambourin-vic-3024', 'maribyrnong-vic-3032', 'maryknoll-vic-3812', 'mccrae-vic-3938', 'mckinnon-vic-3204', 'meadow-heights-vic-3048', 'melbourne-vic-3000', 'melbourne-airport-vic-3045', 'melton-vic-3337', 'melton-south-vic-3338', 'melton-west-vic-3337', 'mentone-vic-3194', 'menzies-creek-vic-3159', 'mernda-vic-3754', 'merricks-vic-3916', 'merricks-beach-vic-3926', 'merricks-north-vic-3926', 'merrimu-vic-3340', 'mickleham-vic-3064'

API CALL VIA PROXY

In [ ]:
for url in urls:
    max_pages = 50
    for x in range(1,max_pages+1):
        if x >= max_pages:
            break
        full_url = 'https://api.webscrapingapi.com/v1?api_key=35mtp7S9Io1GqEja5Up2GocPBGBv5Afr&url=' +urllib.parse.quote_plus(url+str(x))
        print(full_url)
        html_text = requests.get(full_url).text
        soup = BeautifulSoup(html_text,'lxml')
        if x ==1:
            try:
                max_pages = math.ceil(int(soup.find(attrs = {'data-testid':'summary'}).text.split(' ')[0])/20)
                print(str(max_pages) + ' is the number of pages found')
                if max_pages > 50:
                    print(str(max_pages)+ ' is the number of max pages found but we are capping it to 50')
                    max_pages = 50
            except:
                max_pages = 2
        
                
            
        houses = soup.find_all('li',class_= 'css-1qp9106')

        for house in houses:

            # Extract Data Sold & Method of Sale
            date_method = house.find('span', class_='css-1nj9ymt').text
            date_method_split = date_method.split(" ")
            date = f"{date_method_split[-3]} {date_method_split[-2]} {date_method_split[-1]}"
            if "auction" in date_method:
                method = 'auction'
            elif "private treaty" in date_method:
                method = 'private treaty'
            else:
                method = 'Another Method'

            # Extract Address details
            try:
    
                street = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line1'}).text.replace(',','')
                
            except:
                print('ISSUE FOUND WITH STREET')
                street = 'No Street Found'
            
            try:
                 address = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line2'}).text
                 address_split = address.split(' ')
                 postcode = address_split[-1]
                 state = address_split[-2]
                 if len(address_split) == 4:
                    suburb = f"{address_split[0]} {address_split[1]}"
                 elif len(address_split) == 5:
                    suburb = f"{address_split[0]} {address_split[1]} {address_split[2]}"
                 else:
                    suburb = address_split[0]
                
            except:
                print('ISSUE FOUND WITH ADDRESS')
            
               
            

            
            price = house.find('p', class_='css-mgq8yx').text.replace(' price from APM PriceFinder','').replace('$','').replace(',','')
            type = house.find('span', class_='css-693528').text.replace('/ Unit / Flat','')

            try:
                features = house.find('div', class_ ='css-18biwo', attrs = {'data-testid':'property-features-wrapper'})
                bedrooms = features.span.span.text[0]
                if bedrooms == '−' or bedrooms is None:
                    bedrooms = 0
                bathrooms = features.span.next_sibling.span.text[0]
                if bathrooms == '−' or bathrooms is None:
                    bathrooms = 1
                cars = features.span.next_sibling.next_sibling.span.text[0]
                if cars == '−':
                    cars = 0
                if len(list(features.children)) == 4:
                    area = features.span.next_sibling.next_sibling.next_sibling.span.text
                else:
                    area = 0
            except:
                bedrooms = 0
                bathrooms = 0
                cars = 0
                area = 0
                print(f"No features found for {street} {address}")

            agent = house.find('div', class_='css-6yavch',attrs={'data-testid':'listing-card-lazy-image'})
            if agent is None:
                agent = 'UNKNOWN'
            elif agent.img['alt'][0] == 'P':
                agent = 'UNKNOWN'
            else:
                agent = agent.img['alt'].replace('Logo for ','')
            
            
            
        
            house_details = {
                'Street' : street,
                'Address' : address,
                'Suburb' : suburb,
                'State' : state,
                'Postcode' : postcode,
                'Type' : type,
                'Date': date,
                'Method' : method,
                'Price' : price,
                'Bedrooms': bedrooms,
                'Bathrooms': bathrooms,
                'Cars' : cars,
                'Area' : area,
                'Agent' : agent
            }
            if (bedrooms == 0 and bathrooms == 0 and cars == 0 and area == 0) or street == 'No Street Found':
                print(f"{street} {address} house was NOT added")
            else:
                house_list.append(house_details)
               
        print(str(len(house_list))+' houses added so far')
        if len(house_list) > 100:
            df = pd.DataFrame(house_list)
            df.to_csv(f"../MLrealestate/data/{city}/{city}_raw.csv",mode='a', header = False, index = False)
            house_list = []



SAVING PANDAS DF TO CSV. PLEASE NOTE: WE ARE APPENDING TO CSV

In [ ]:


df = pd.DataFrame(house_list)
df.to_csv(f"../MLrealestate/data/{city}/{city}_raw.csv",mode='a', header = False, index = False)
df.tail(6)

Creating Latitude & Longitude columns from address

In [3]:

df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_raw.csv")

def get_coordinates(street, suburb):
    loc = Nominatim(user_agent="GetLoc")

    try:
        if {street} == 'No Street Found':
            getLoc = loc.geocode(f"{suburb},Australia")
        else:
            getLoc = loc.geocode(f"{street},{suburb},Australia")
        lat = getLoc.latitude
        long = getLoc.longitude
    except:
        try:
            getLoc = loc.geocode(f"{suburb},Australia")
            lat = getLoc.latitude
            long = getLoc.longitude
        except:
            print(f"{street},{suburb},Australia was not able to find lat & long")
            lat = -37.7618477
            long = 144.9805318
    return lat,long


vect_get_coordinates = np.vectorize(get_coordinates)

completed = 0

for i in range(0,len(df.index),1000):
    df_seg = df[i:i+1000]
    df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
    df_seg.to_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv",mode='a',header = False,index=False)
    completed = completed + len(df_seg)
    print(f"PROCESSED: {completed} RECORDS OUT OF TOTAL {len(df.index)}")
    print(f"PROGRESS: {completed/len(df.index)} % COMPLETED")

    

17 Inverness Avenue ,DERRIMUT VIC 3030,Australia was not able to find lat & long
32 Britannia way ,LILYDALE VIC 3140,Australia was not able to find lat & long
68 Bastow Road ,LILYDALE VIC 3140,Australia was not able to find lat & long


/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)


PROCESSED: 1000 RECORDS OUT OF TOTAL 117523
PROGRESS: 0.008508972711724513 % COMPLETED
32 Noremac Grove ,LYNDHURST VIC 3975,Australia was not able to find lat & long


/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)


PROCESSED: 2000 RECORDS OUT OF TOTAL 117523
PROGRESS: 0.017017945423449025 % COMPLETED


...

/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)


PROCESSED: 53000 RECORDS OUT OF TOTAL 117523
PROGRESS: 0.4509755537213992 % COMPLETED


/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)


PROCESSED: 54000 RECORDS OUT OF TOTAL 117523
PROGRESS: 0.4594845264331237 % COMPLETED
57 Pinoak Drive ,YARRA GLEN VIC 3775,Australia was not able to find lat & long
PROCESSED: 54523 RECORDS OUT OF TOTAL 117523
PROGRESS: 0.46393471916135565 % COMPLETED


/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
/tmp/ipykernel_415/2981532999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)


In [14]:
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv")

len(df[df['Latitude'] == -37.7618477])

0

In [12]:
def get_coordinates(street, suburb):
    loc = Nominatim(user_agent="GetLoc")

    try:
        if {street} == 'No Street Found':
            getLoc = loc.geocode(f"{suburb},Australia")
        else:
            getLoc = loc.geocode(f"{street},{suburb},Australia")
        lat = getLoc.latitude
        long = getLoc.longitude
    except:
        try:
            getLoc = loc.geocode(f"{suburb},Australia")
            lat = getLoc.latitude
            long = getLoc.longitude
        except:
            print(f"{street},{suburb},Australia was not able to find lat & long")
            lat = -37.7618477
            long = 144.9805318
    return lat,long

df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv")

def fix_lat_lon(row):
    if row['Latitude'] == -37.7618477:
       try:
            row['Latitude'],row['Longitude'] = get_coordinates(row['Street'], row['Suburb'])
           
            print(f"fixed {row['Street']} new lat = {row['Latitude']}  new long= {row['Longitude']}")
       except:
            print(f"Unable to fix {row['Street']}")
    return row


df = df.apply(fix_lat_lon, axis=1)
df.head()      




fixed 108/3 Faulkner Street  new lat = -37.9323275  new long= 145.0373655
fixed 53 South Avenue  new lat = -37.930122  new long= 145.0357656
fixed 5 Delhi Street  new lat = -37.925692  new long= 145.0477795
fixed 9 Bendigo Avenue  new lat = -37.9188039  new long= 145.0373582
fixed 3A Renown Street  new lat = -37.9213809  new long= 145.0465663
fixed 5 Gwendoline Avenue  new lat = -37.9159723  new long= 145.026324
fixed 3 Golf Links Crescent  new lat = -37.9864541  new long= 145.1399716
fixed 300 Hazeldene Road  new lat = -37.8415589  new long= 145.6435486
fixed 5 Dickens Street  new lat = -37.8484727  new long= 145.070148
fixed 16 Atkins Avenue  new lat = -37.8488025  new long= 145.0655029
fixed 16 Goodwin Street  new lat = -37.8507757  new long= 145.0701675
fixed 3/40 Maude Avenue  new lat = -37.7028595  new long= 144.9354002
fixed 4/40 Belair Avenue  new lat = -37.7044429  new long= 144.9142018
fixed 162 Greenvale Drive  new lat = -37.6463365  new long= 144.8981351
fixed 50 Napoli Cir

,Street,Address,Suburb,State,Postcode,Type,Date,Method,Price,Bedrooms,Bathrooms,Cars,Area,Agent,Latitude,Longitude
0,6 Clifton Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,25-Jun-22,private treaty,1810000,4,2,3,535m²,MATTHEWS.,-37.761302,144.891030
1,14 Batman Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,21-May-22,auction,2101000,4,3,2,650m²,Woodards Ascot Vale,-37.755418,144.893808
2,12 Batman Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,9-Apr-22,auction,1900000,4,2,2,650m²,Nelson Alexander Essendon,-37.755418,144.893808
3,24 Clifton Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,1-Apr-22,auction,2000000,3,2,5,640m²,Barry Plant Essendon,-37.761302,144.891030
4,34 Caroline Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,26-Mar-22,auction,2981000,4,3,3,"1,017m²",Ray White Sunbury Pty Ltd,-37.759417,144.892162


In [13]:
df.to_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv", header = True, index = False)

CALCULATE DISTANCE FROM CITY by vectorization function and using Latitude and longitude

In [15]:


df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv")
la2,lo2 = get_coordinates('No Street Found', city)

def calc_distance(la1,lo1):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(la1)
    lon1 = radians(lo1)
    lat2 = radians(la2)
    lon2 = radians(lo2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
   

vec_cal_distance = np.vectorize(calc_distance)
df['Distance']  = vec_cal_distance(df.Latitude, df.Longitude)
df.to_csv(f"../MLrealestate/data/{city}/{city}_distance.csv",header = True,index=False)
df.head(20)

,Street,Address,Suburb,State,Postcode,Type,Date,Method,Price,Bedrooms,Bathrooms,Cars,Area,Agent,Latitude,Longitude,Distance
0,6 Clifton Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,25-Jun-22,private treaty,1810000,4,2,3,535m²,MATTHEWS.,-37.761302,144.891030,8.651315
1,14 Batman Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,21-May-22,auction,2101000,4,3,2,650m²,Woodards Ascot Vale,-37.755418,144.893808,8.941133
2,12 Batman Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,9-Apr-22,auction,1900000,4,2,2,650m²,Nelson Alexander Essendon,-37.755418,144.893808,8.941133
3,24 Clifton Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,1-Apr-22,auction,2000000,3,2,5,640m²,Barry Plant Essendon,-37.761302,144.891030,8.651315
4,34 Caroline Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,26-Mar-22,auction,2981000,4,3,3,"1,017m²",Ray White Sunbury Pty Ltd,-37.759417,144.892162,8.723874
5,3 Beatrice Avenue,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,15-Mar-22,private treaty,2540000,5,4,4,613m²,Jellis Craig Moonee Valley,-37.753983,144.889504,9.317378
6,1 Vida Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,4-Mar-22,private treaty,1855000,4,2,2,569m²,McDonald Upton,-37.755976,144.898742,8.604382
7,37 Clifton Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,19-Feb-22,auction,1945000,4,3,2,810m²,Barry Plant Essendon,-37.761302,144.891030,8.651315
8,104 Fawkner Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,19-Feb-22,auction,2007000,3,2,1,481m²,Barry Plant Essendon,-37.757372,144.896145,8.641993
9,5 Muldowney Street,ABERFELDIE VIC 3040,ABERFELDIE,VIC,3040,House,5-Feb-22,auction,2501000,4,2,3,482m²,Nelson Alexander Essendon,-37.758229,144.896914,8.526209


Clean up Area by removing commas,m2, and converting Ha to m2

In [ ]:
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_distance.csv")

def convert_area(area):
    if ',' in area:
        area = area.replace(',','')
    if 'm²' in area:
        area = area.replace('m²','')
    if 'ha' in area:
        area = 10000*float(area.replace('ha',''))
    return float(area)
   

vec_convert_area = np.vectorize(convert_area)
df['Area']  = vec_convert_area(df.Area)


df.to_csv(f"../MLrealestate/data/{city}/{city}_area.csv",header = True,index=False)

In [ ]:
import plotly.express as px
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_distance.csv")
fig = px.scatter_mapbox(df,lat = df['Latitude'],lon = df['Longitude'],zoom =8,color = df['Price'],size = df['Price'], text =df.Street+ df.Address, width = 1100, height = 800, title = city + ' House Prices Map')
fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(margin = {'r':0 , 't': 50, 'l':0 , 'b':10})
fig.show()

Send Email of the collected & transformed data

In [ ]:

emailsend.run(f"../MLrealestate/data/{city}/{city}_area.csv")